<a href="https://colab.research.google.com/github/max-enterme/Naga-Calc/blob/main/NAGA%E6%8E%A1%E7%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 採点
url = "" #@param {type:"string"}
import json
import requests
from os import mkdir
from bs4 import BeautifulSoup 
from google.colab import files

def parseNagaReport(url):
  html = requests.get(url)
  soup = BeautifulSoup(html.content, "html.parser")
  script = soup.select("script")[0].text
  scriptPerLine = script.splitlines()
  return {
    "pred": json.loads(scriptPerLine[1].split("=")[1]),
    "playerInfo": json.loads(scriptPerLine[2].split("=")[1].replace("'", "\"")),
    "haihuId": scriptPerLine[3].split("=")[1].strip("  \""),
    "nagaVersion": scriptPerLine[4].split("=")[1].strip("  \""),
    "gameType": scriptPerLine[5].split("=")[1].strip()
  }

pai2index = {
  "1m":0, "2m":1, "3m":2, "4m": 3, "5m":4, "5mr":4, "6m":5, "7m": 6, "8m":7, "9m":8,
  "1p":9, "2p":10, "3p":11, "4p":12, "5p":13, "5pr":13, "6p":14, "7p":15, "8p":16, "9p":17,
  "1s":18, "2s":19, "3s":20, "4s":21, "5s":22, "5sr":22, "6s":23, "7s":24, "8s":25, "9s":26,
  "E":27, "S":28, "W":29, "N":30, "P":31, "F":32, "C":33
}

report = parseNagaReport(url)

# ここから算出処理を実装していく
# 参考サイト: https://note.com/suoh2009/n/n7eb51e50b4e9

# 出したい指標
# - 総打数
# - 一致数
# - 不一致数
# - 一致率
# - 不一致率
# - 悪手率

# 副露の情報: 一個前の打牌情報の "huro" にある
#  - 0: しない
#  - 1: チー (上の両面)
#  - 2: チー (カンチャン)
#  - 3: チー (下の両面)
#  - 4: ポン
#  - 5: 大明槓(?)
# 暗槓の情報: 実行したら、type が "ankan" で入ってくる
# 暗槓するかどうかの判断は、type = tsumo の時に "kan" に入ってくる

def create_info():
    return {
        "total_count": 0,
        "total_score_diff": 0,
        "same_count": 0
    }

actor_logs = {
    0: "",
    1: "",
    2: "",
    3: "",
}

actor_infos = {
    0: create_info(),
    1: create_info(),
    2: create_info(),
    3: create_info(),
}

for actions in report["pred"]:
  for action in actions:
    msg = action["info"]["msg"]
    if "actor" in msg:
      if "real_dahai" in msg and msg["real_dahai"] != None and "pred_dahai" in msg:
        actor = msg["actor"]
        if not msg["reached"]:
          dahai_score = action["dahai_pred"][pai2index[msg["real_dahai"]]]
          pred_score = action["dahai_pred"][pai2index[msg["pred_dahai"]]]
          actor_infos[actor]["total_score_diff"] += pred_score - dahai_score
          actor_infos[actor]["total_count"] += 1
          if msg["real_dahai"] == msg["pred_dahai"]:
            actor_infos[actor]["same_count"] += 1
          
        actor_logs[actor] += f'{msg["type"]}:{msg["pai"]}, 打牌:{msg["real_dahai"]}, 推奨:{msg["pred_dahai"]} => {dahai_score}:{pred_score}, {sum(action["dahai_pred"])}, 立直: {msg["reached"]}\n'

for number, info in actor_infos.items():
  print("--------------------------------------------------------")
  print(f'プレイヤー名: {report["playerInfo"]["name"][number]}')
  print(f'打数: {info["total_count"]}, 一致数: {info["same_count"]}, 一致率: {round(info["same_count"]/info["total_count"]*100,2)}%')
  print(f'打牌評価: {round(100 - info["total_score_diff"] / 100 / info["total_count"], 2)}')
  print(f'総合評価: 実装途中')

In [ ]:
#@title 詳細レポート保存 (実装途中)
!rm -rfd result | true
!rm artifacts.zip | true
!mkdir result
open("result/pred.json", "wt").write(json.dumps(report["pred"], indent=4))
open("result/meta.json", "wt").write(
    json.dumps({
        "playerInfo": report["playerInfo"],
        "haihuId": report["haihuId"],
        "nagaVersion": report["nagaVersion"],
        "gameType": report["gameType"]
    }, ensure_ascii = False, indent=4)
)

open("result/actor_0.txt", "wt").write(actor_logs[0])
open("result/actor_1.txt", "wt").write(actor_logs[1])
open("result/actor_2.txt", "wt").write(actor_logs[2])
open("result/actor_3.txt", "wt").write(actor_logs[3])

!zip -j -r artifacts.zip result
files.download("artifacts.zip")